In [ ]:
%pip install -q cryptography mediapy

# Some imports
from os import urandom
import struct
from binascii import hexlify
import numpy as np
import mediapy as media

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
# from cryptography.hazmat.backends import default_backend

# Using ChaCha20

In [ ]:
# key is 256 bits (32 bytes)
key = urandom(32)

# nonce is a random value (64 bits) concatenated to a counter (64 bits)
# note that IETF standard has a different split (96/32) 
nonce = urandom(8)
counter = 0
full_nonce = struct.pack("<Q", counter) + nonce

algorithm = algorithms.ChaCha20(key, full_nonce)
cipher = Cipher(algorithm, mode=None)
encryptor = cipher.encryptor()
decryptor = cipher.decryptor()

In [ ]:
m = b"a secret message"
ct = encryptor.update(b"a secret message")
hexlify(ct)

In [ ]:
decryptor.update(ct)

## Encrypt an image

In [ ]:
key = urandom(32)
nonce = urandom(8)
counter = 0
full_nonce = struct.pack("<Q", counter) + nonce

algorithm = algorithms.ChaCha20(key, full_nonce)
cipher = Cipher(algorithm, mode=None)
encryptor = cipher.encryptor()
decryptor = cipher.decryptor()

In [ ]:
IMAGE = 'https://raw.githubusercontent.com/gverticale/network-security-and-cryptography/master/tux_gray.png'

tux_png = media.read_image(IMAGE)
media.show_image(tux_png,height=480, title='Tux')

In [ ]:
# Keep only luminance and serialize the image
# The plaintext is a string of bytes
tux = tux_png[:,:,1] * 255
tux = tux.astype(np.uint8)
pt = tux.tobytes()

In [ ]:
ct = encryptor.update(pt)

In [ ]:
# Reshape the ciphertext into a matrix of bytes 
# Show the ciphertext
encrypted_tux = np.frombuffer(ct, dtype = np.uint8).reshape(tux.shape)
media.show_image(encrypted_tux,height=480, title='Encrypted Tux')

## Encrypt an image with the same key and nonce (WRONG!)

In [ ]:
algorithm2 = algorithms.ChaCha20(key, full_nonce)
cipher2 = Cipher(algorithm, mode=None)
encryptor2 = cipher.encryptor()
decryptor2 = cipher.decryptor()

In [ ]:
def create_checkerboard_matrix(rows, cols, square_size):
    # Create an empty matrix filled with zeros
    matrix = np.zeros((rows, cols), dtype=int)
    
    # Iterate over each square
    for i in range(0, rows, square_size):
        for j in range(0, cols, square_size):
            # Set ones in alternating squares
            if (i // square_size + j // square_size) % 2 == 0:
                matrix[i:i+square_size, j:j+square_size] = 255
    
    return matrix

im_checkerboard = np.uint8(create_checkerboard_matrix(1024,869,16))
media.show_image(im_checkerboard,height=640)

In [ ]:
ct2=encryptor2.update(im_checkerboard.reshape(-1))

In [ ]:
encrypted_board = np.frombuffer(ct2, dtype = np.uint8).reshape(tux.shape)
media.show_image(encrypted_board,height=640,title='Encrypted board')

What happens if we xor the ciphertexts?  

In [ ]:
# Calculate the XOR between the ciphertexts
ciphertext_xor = np.bitwise_xor(encrypted_tux,encrypted_board)
difference = np.frombuffer(ciphertext_xor, dtype = np.uint8).reshape(tux.shape)
media.show_image(ciphertext_xor,height=480, title='Xored')

In [ ]:
media.compare_images([tux_png,ciphertext_xor])

# Lab Activity: known plaintext attack
* Assume you know the checkerboard plaintext. Can you recover tux from the ciphertext?
* What happens if the counter of the checkerboard's nonce is not 0 but 5? Can you recover something? Hint: use `np.roll`